In [46]:
function parse_values_parts(part_in)
    values = split(part_in, ',')
    out = Dict('x' => parse(Int, split(values[1], '=')[2]), 'm' => parse(Int, split(values[2], '=')[2]), 'a' => parse(Int, split(values[3], '=')[2]), 's' => parse(Int, split(values[4], '=')[2]), 'n' => 0)
    return out
end

parse_values_parts (generic function with 1 method)

In [47]:
function parse_conditions(sequence_wfl)
    out_seq = []

    for a_condition in split(sequence_wfl, ',')
        el = split(a_condition, ':')
        if length(el) == 1
            # Default true condition
            append!(out_seq, [['n', '>', -1, a_condition]])
        else 
            append!(out_seq, [[el[1][1], el[1][2], parse(Int, el[1][3:end]), el[2]]])
        end
    end
    return out_seq

end

parse_conditions (generic function with 1 method)

In [48]:
parse_conditions("s<537:gd,x>2440:R,A")

3-element Vector{Any}:
 Any['s', '<', 537, "gd"]
 Any['x', '>', 2440, "R"]
 Any['n', '>', -1, "A"]

In [51]:
function evaluate_conditions(part, wfl)
    keep_going = true
    current_condition_nb = 1
    # println(wfl)
    out = "in"
    while keep_going
        data_name, data_type, comp_value, out = wfl[current_condition_nb]
        # println(wfl[current_condition_nb])
        if data_type == '>'
            keep_going = (part[data_name] < comp_value)
        else
            keep_going = (part[data_name] > comp_value)
        end
        current_condition_nb += 1
        # println(keep_going)
    end
    out
        
end

evaluate_conditions (generic function with 1 method)

In [54]:
(xmas_sum) = open("19_1.txt") do f
    x_s = 0

    parts = Dict()
    rules = Dict()
    sorting = Dict("A" => [], "R" => [])

    l = readline(f)
    while (l != "")
        values = split(l, ['{', '}'])
        rules[values[1]] = parse_conditions(values[2])
        l = readline(f)
    end

    # println(rules)

    partnb = 1

    for l in eachline(f) 
        parts[partnb] = parse_values_parts(split(l, ['{', '}'])[2])
        wfl_name = "in"
        while !(wfl_name in ["A", "R"])
            wfl_name = evaluate_conditions(parts[partnb], rules[wfl_name])
        end
        append!(sorting[wfl_name], partnb)
        partnb += 1
    end

    # println(parts)

    for i in sorting["A"]
        x_s += parts[i]['x']
        x_s += parts[i]['m']
        x_s += parts[i]['a']
        x_s += parts[i]['s']
    end
    
    (x_s)
end

434147

In [11]:
blah = []

Any[]

In [12]:
append!(blah, ["a_string"])

1-element Vector{Any}:
 "a_string"

In [13]:
append!(blah, 123)

2-element Vector{Any}:
    "a_string"
 123

In [17]:
append!(blah, [1,2,3])

5-element Vector{Any}:
    "a_string"
 123
   1
   2
   3

In [18]:
append!(blah, [[1,2,3]])

6-element Vector{Any}:
    "a_string"
 123
   1
   2
   3
    [1, 2, 3]

In [72]:
(xmas_sum) = open("19_1.txt") do f
    x_s = 0

    # parts = Dict()
    rules = Dict()
    sorting = Dict("A" => [], "R" => [])

    l = readline(f)
    while (l != "")
        values = split(l, ['{', '}'])
        rules[values[1]] = parse_conditions(values[2])
        l = readline(f)
    end

    all_ranges = [Dict("next" => "in", 'x' => [1,4000], 'm' => [1,4000], 'a' => [1,4000], 's' => [1,4000])]

    while length(all_ranges) > 0 
        # println("Ranges are $all_ranges")
        current_range = pop!(all_ranges)
        # println("And now they are $all_ranges")
        if current_range["next"] == "A"
            x_s += (current_range['x'][2]-current_range['x'][1]+1)*(current_range['m'][2]-current_range['m'][1]+1)*(current_range['a'][2]-current_range['a'][1]+1)*(current_range['s'][2]-current_range['s'][1]+1)
            continue
        elseif current_range["next"] == "R"
            continue
        else
            all_conditions = rules[current_range["next"]]
            #all_conditions = parse_conditions(current_rule)
            for a_condition in all_conditions
                # println("Updating ranges:  $all_ranges")
                (data_name, data_type, comp_value, next) = a_condition
                # println(a_condition)
                if data_name == 'n'
                    current_range["next"] = next
                    push!(all_ranges, deepcopy(current_range))
                elseif data_type == '<'
                    if current_range[data_name][2] < comp_value
                        # All data range is ok
                        current_range["next"] = next
                        push!(all_ranges, current_range)
                        continue
                    elseif current_range[data_name][1] >= comp_value
                        # Current condition is not verified
                    else
                        # Domain splitting case
                        old_range = current_range[data_name]
                        old_next = current_range["next"]
                        current_range["next"] = next
                        current_range[data_name] = [old_range[1], comp_value-1]
                        push!(all_ranges, deepcopy(current_range))
                        current_range[data_name] = [comp_value, old_range[2]]
                        current_range["next"] = old_next
                    end
                else
                    if current_range[data_name][2] <= comp_value
                        # Current condition is not verified
                    elseif current_range[data_name][1] > comp_value
                        # All data range is ok
                        current_range["next"] = next
                        push!(all_ranges, current_range)
                        continue
                    else
                        # Domain splitting case
                        old_range = current_range[data_name]
                        old_next = current_range["next"]
                        current_range["next"] = next
                        current_range[data_name] = [comp_value+1, old_range[2]]
                        push!(all_ranges, deepcopy(current_range))
                        current_range[data_name] = [old_range[1], comp_value]
                        current_range["next"] = old_next
                    end
                end
            end 
        end
    end
    
    (x_s)
end

136146366355609